In [1]:
unmosaicked_input = 'out/DelawareRiverBasin/Run11_11_2020/'
enduser_cog_output = 'enduser/DelawareRiverBasin/drb150_nofunswitch/'
year='1954'
product='netet'
cmd_opt = '-i ' + unmosaicked_input + ' -o ' + enduser_cog_output + ' -y ' + year + ' ' + product + ' dummy'

NUM_CONTAINERS = 26 # 40 is too high, maybe 25

MAX_LOAD_LEVEL = 220
MIN_MEMORY_AVAILABLE = 4

MAX_CONCURRENT_CONTAINERS = NUM_CONTAINERS

#start_year = 1950
start_year = 1991
end_year = 2099

In [2]:
import docker

In [3]:
# get client
client = docker.from_env()

In [4]:
running_containers = client.containers.list()
for c in running_containers:
    print(c.name)

pie_pie-service_1
brave_hofstadter


In [5]:
def start_container(client, docker_image, docker_full_cmd):
    container = client.containers.run(docker_image, docker_full_cmd, detach=True)
    print ( "CONTAINER is ", container.name)
    return(container)

In [6]:
def start_etm(year):
    global unmosaicked_input
    global enduser_cog_output
    global product
    
    cmd_opt = '-i ' + unmosaicked_input + ' -o ' + enduser_cog_output + ' -y ' + year + ' ' + product + ' dummy'
    print(cmd_opt)
    cmd = 'python3 api_etm.py '
    full_cmd = cmd + cmd_opt
    #print(full_cmd)
    docker_image =  "tbutzer/etm_docker_image"
    #print(docker_image)
    c = start_container(client, docker_image, full_cmd)
    print("real name is", c.name)
    print("==="*30)

In [7]:
for yeari in range(start_year, start_year+NUM_CONTAINERS+1):
    print(yeari)
    year=str(yeari)
    start_etm(year)

1991
-i out/DelawareRiverBasin/Run11_11_2020/ -o enduser/DelawareRiverBasin/drb150_nofunswitch/ -y 1991 netet dummy
CONTAINER is  stupefied_davinci
real name is stupefied_davinci
1992
-i out/DelawareRiverBasin/Run11_11_2020/ -o enduser/DelawareRiverBasin/drb150_nofunswitch/ -y 1992 netet dummy
CONTAINER is  practical_swirles
real name is practical_swirles
1993
-i out/DelawareRiverBasin/Run11_11_2020/ -o enduser/DelawareRiverBasin/drb150_nofunswitch/ -y 1993 netet dummy
CONTAINER is  romantic_almeida
real name is romantic_almeida
1994
-i out/DelawareRiverBasin/Run11_11_2020/ -o enduser/DelawareRiverBasin/drb150_nofunswitch/ -y 1994 netet dummy
CONTAINER is  admiring_cerf
real name is admiring_cerf
1995
-i out/DelawareRiverBasin/Run11_11_2020/ -o enduser/DelawareRiverBasin/drb150_nofunswitch/ -y 1995 netet dummy
CONTAINER is  nifty_dijkstra
real name is nifty_dijkstra
1996
-i out/DelawareRiverBasin/Run11_11_2020/ -o enduser/DelawareRiverBasin/drb150_nofunswitch/ -y 1996 netet dummy
CONTA

In [8]:
running_containers = client.containers.list()
for c in running_containers:
    if len(c.attrs['Args']) >7 :
        print(c.name, c.attrs['Args'][6], c.attrs['Args'][7])

fervent_galois 2017 netet
crazy_taussig 2016 netet
goofy_hermann 2015 netet
xenodochial_babbage 2014 netet
condescending_mayer 2013 netet
gallant_montalcini 2012 netet
brave_visvesvaraya 2011 netet
brave_mendel 2010 netet
trusting_pascal 2009 netet
focused_heyrovsky 2008 netet
condescending_faraday 2007 netet
ecstatic_dijkstra 2006 netet
distracted_diffie 2005 netet
amazing_elgamal 2004 netet
xenodochial_haibt 2003 netet
upbeat_banzai 2002 netet
lucid_saha 2001 netet
xenodochial_shockley 2000 netet
interesting_gould 1999 netet
funny_bartik 1998 netet
elastic_hodgkin 1997 netet
goofy_williams 1996 netet
nifty_dijkstra 1995 netet
admiring_cerf 1994 netet
romantic_almeida 1993 netet
practical_swirles 1992 netet
stupefied_davinci 1991 netet


- https://medium.com/the-andela-way/machine-monitoring-tool-using-python-from-scratch-8d10411782fd

# Should maybe use os.getloadavg()[0]

In [9]:
import os
def return_cpu_load():
    cpu_load = [x / os.cpu_count() * 100 for x in os.getloadavg()][-1]
    return cpu_load

In [10]:
os.cpu_count()

8

In [11]:
import subprocess
import re

def _return_mem_stat():


    # Memory usage
    total_ram = subprocess.run(['free', '-h'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    used_free_shared_buf_avail = total_ram.split('\n')[1]
    
    return  used_free_shared_buf_avail

def return_available_memory():
    used_free_shared_buf_avail = _return_mem_stat()
    a = re.split('\s+', used_free_shared_buf_avail)
    available_memory = a[3].split('G')[0]
    return float(available_memory)

In [12]:
_return_mem_stat()

'Mem:           31Gi       3.0Gi        25Gi       2.0Mi       3.0Gi        27Gi'

In [13]:
available_memory = return_available_memory()
available_memory

25.0

In [14]:
os.getloadavg()

(4.47, 4.46, 7.08)

In [15]:
def return_num_containers():
    global client
    running_containers = client.containers.list()
    return(len(running_containers))

In [16]:
import time
def event_loop(year_to_process, end_year):
    
    
    while year_to_process <= end_year:
        time.sleep(30)

        mem_avail = return_available_memory()
        cpu_load = return_cpu_load()
        num_running_containers = return_num_containers()
        print(mem_avail, cpu_load, num_running_containers)

        cpu = False
        if (cpu_load < MAX_LOAD_LEVEL):
            print("CPU is FINE")
            cpu=True

        mem = False
        if mem_avail > MIN_MEMORY_AVAILABLE:
            mem = True

        containers = False
        if num_running_containers < MAX_CONCURRENT_CONTAINERS:
            containers = True
        

        if (mem and cpu and containers):
            print("OK to Launch")
            start_etm(year_to_process) ### start mosaic container
            print("starting year", year_to_process)
            year_to_process = year_to_process + 1

    
    

In [ ]:
cyear = start_year + NUM_CONTAINERS + 1
event_loop(cyear, end_year)

19.0 92.125 29
CPU is FINE
18.0 94.5 29
CPU is FINE
18.0 96.0 29
CPU is FINE
18.0 97.875 29
CPU is FINE
18.0 103.49999999999999 29
CPU is FINE
16.0 103.62499999999999 29
CPU is FINE
16.0 106.25 29
CPU is FINE
14.0 108.25 29
CPU is FINE
10.0 112.25 29
CPU is FINE
12.0 112.99999999999999 29
CPU is FINE
18.0 116.625 29
CPU is FINE
